In [2]:
#  ___________________________________________________________________________
#
#  Surrogate vs. Implicit: Experiments comparing nonlinear optimization
#  formulations
#
#  Copyright (c) 2023. Triad National Security, LLC. All rights reserved.
#
#  This program was produced under U.S. Government contract 89233218CNA000001
#  for Los Alamos National Laboratory (LANL), which is operated by Triad
#  National Security, LLC for the U.S. Department of Energy/National Nuclear
#  Security Administration. All rights in the program are reserved by Triad
#  National Security, LLC, and the U.S. Department of Energy/National Nuclear
#  Security Administration. The Government is granted for itself and others
#  acting on its behalf a nonexclusive, paid-up, irrevocable worldwide license
#  in this material to reproduce, prepare derivative works, distribute copies
#  to the public, perform publicly and display publicly, and to permit others
#  to do so.
#
#  This software is distributed under the 3-clause BSD license.
#  ___________________________________________________________________________

## 1. Import necessary packages. 
- The IDAES version being used is 2.0

In [3]:
import numpy as np
import random 
import pandas as pd
import pyomo.environ as pyo
from idaes.core.util.exceptions import InitializationError
from pyomo.environ import (
    Constraint,
    Var,
    ConcreteModel,
    Expression,
    Objective,
    TransformationFactory,
    value,
    units as pyunits,
)
from pyomo.network import Arc

from idaes.core import FlowsheetBlock
from idaes.models.properties.modular_properties import GenericParameterBlock
from idaes.models.unit_models import (
    Feed,
    Mixer,
    Compressor,
    Heater,
    GibbsReactor,
    Product,
)

from idaes.core.solvers import get_solver
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.models_extra.power_generation.properties.natural_gas_PR import get_prop

## 2. Generate samples for the Gibbs Reactor 

- Assumption: Outlet temperature and pressure of the Gibbs reactor are fixed. 

In [4]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
thermo_props_config_dict = get_prop(components=["CH4", "H2O", "H2", "CO", "CO2"])
m.fs.thermo_params = GenericParameterBlock(**thermo_props_config_dict)

m.fs.R101 = GibbsReactor(
    property_package=m.fs.thermo_params,
    has_heat_transfer=True,
    has_pressure_change=True,
)

Fin_min = 1.027 # mol/s
Tin_min = 401 # K
Fin_max = 3.989 # mol/s
Tin_max = 599 # K

num_samples = 10

df = {'T_in (K)':[], 'F_in (K)':[], 'Heat Duty (kW)':[], 'F_out (K)':[], 
      'T_out (K)':[], 'H2':[], 'H2O':[], 'CO2':[], 'CO':[], 'CH4':[]}

for i in range(num_samples):
      try:
            Tin = random.uniform(Tin_min, Tin_max)
            Fin = random.uniform(Fin_min, Fin_max)
            m.fs.R101.inlet.flow_mol.fix(Fin)
            m.fs.R101.inlet.temperature.fix(Tin)
            m.fs.R101.inlet.pressure.fix(203395.9 * pyunits.Pa)
            m.fs.R101.inlet.mole_frac_comp[0, 'CH4'].fix(0.2499)
            m.fs.R101.inlet.mole_frac_comp[0, 'H2'].fix(1e-20)
            m.fs.R101.inlet.mole_frac_comp[0, 'CO'].fix(1e-20)
            m.fs.R101.inlet.mole_frac_comp[0, 'CO2'].fix(1e-20)
            m.fs.R101.inlet.mole_frac_comp[0, 'H2O'].fix(0.75)
            m.fs.R101.outlet.pressure.fix(137895 * pyunits.Pa)  
            m.fs.R101.outlet.temperature.fix(1060.93 * pyunits.K)
            m.fs.R101.initialize()
            solver = get_solver()
            results = solver.solve(m, tee=False)
            df['T_in (K)'].append(Tin)
            df['F_in (K)'].append(Fin)
            df['Heat Duty (kW)'].append(pyo.value(m.fs.R101.heat_duty[0]))
            df['F_out (K)'].append(pyo.value(m.fs.R101.outlet.flow_mol[0]))
            df['T_out (K)'].append(pyo.value(m.fs.R101.outlet.temperature[0]))
            df['H2'].append(pyo.value(m.fs.R101.outlet.mole_frac_comp[0, "H2"]))
            df['H2O'].append(pyo.value(m.fs.R101.outlet.mole_frac_comp[0, "H2O"]))
            df['CO2'].append(pyo.value(m.fs.R101.outlet.mole_frac_comp[0, "CO2"]))
            df['CO'].append(pyo.value(m.fs.R101.outlet.mole_frac_comp[0, "CO"]))
            df['CH4'].append(pyo.value(m.fs.R101.outlet.mole_frac_comp[0, "CH4"]))
      except InitializationError:
            print(f"Initialization error at {Tin,Fin}.")
            continue
     
        
df = pd.DataFrame(df)

2023-05-29 22:07:49 [INFO] idaes.init.fs.R101.control_volume.properties_in: Starting initialization
2023-05-29 22:07:49 [INFO] idaes.init.fs.R101.control_volume.properties_in: Property initialization: optimal - Optimal Solution Found.
2023-05-29 22:07:49 [INFO] idaes.init.fs.R101.control_volume.properties_out: Starting initialization
2023-05-29 22:07:49 [INFO] idaes.init.fs.R101.control_volume.properties_out: Property initialization: optimal - Optimal Solution Found.
2023-05-29 22:07:49 [INFO] idaes.init.fs.R101.control_volume: Initialization Complete
2023-05-29 22:07:49 [INFO] idaes.init.fs.R101: Initialization Complete: optimal - Optimal Solution Found
2023-05-29 22:07:50 [INFO] idaes.init.fs.R101.control_volume.properties_in: Starting initialization
2023-05-29 22:07:50 [INFO] idaes.init.fs.R101.control_volume.properties_in: Property initialization: optimal - Optimal Solution Found.
2023-05-29 22:07:50 [INFO] idaes.init.fs.R101.control_volume.properties_out: Starting initialization
2

In [5]:
df # To do: Generate more samples and export the dataframe, which will be used to generate surrogate models.

,T_in (K),F_in (K),Heat Duty (kW),F_out (K),T_out (K),H2,H2O,CO2,CO,CH4
0,420.275669,2.986885,242030.415058,4.470572,1060.93,0.557506,0.27553,0.059588,0.106384,0.000991
1,465.487586,1.606819,127513.335723,2.404981,1060.93,0.557506,0.27553,0.059588,0.106384,0.000991
2,510.977116,1.589597,123401.627870,2.379204,1060.93,0.557506,0.27553,0.059588,0.106384,0.000991
3,478.788713,2.275595,179447.201087,3.405960,1060.93,0.557506,0.27553,0.059588,0.106384,0.000991
4,470.335478,2.657674,210423.132369,3.977830,1060.93,0.557506,0.27553,0.059588,0.106384,0.000991
5,577.948084,2.688845,201679.408150,4.024485,1060.93,0.557506,0.27553,0.059588,0.106384,0.000991
6,597.337524,3.322500,246620.421619,4.972898,1060.93,0.557506,0.27553,0.059588,0.106384,0.000991
7,443.741544,1.523158,122108.047481,2.279762,1060.93,0.557506,0.27553,0.059588,0.106384,0.000991
8,465.067946,1.643563,130455.064396,2.459976,1060.93,0.557506,0.27553,0.059588,0.106384,0.000991
9,453.970501,2.633958,210157.819770,3.942333,1060.93,0.557506,0.27553,0.059588,0.106384,0.000991
